# SVC 
Using the classic ML approach to classify the foods.

In [4]:
# Import libraries
import pandas as pd
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from skimage.io import imread
from skimage.io import ImageCollection
from skimage.transform import resize

ModuleNotFoundError: No module named 'skimage'

In [ ]:
# Import data
ic_train = ImageCollection('AIPI540_proj1_data/raw_data/public_training_set_release_2.0/images/*.jpg')
ic_val = ImageCollection('AIPI540_proj1_data/raw_data/public_validation_set_release_2.0/images/*.jpg')
ic_test = ImageCollection('AIPI540_proj1_data/raw_data/public_testing_set_release_2.0/images/*.jpg')

# Create labels
train_labels = json.load(open('AIPI540_proj1_data/raw_data/public_training_set_release_2.0/annotations.json'))
val_labels = json.load(open('AIPI540_proj1_data/raw_data/public_validation_set_release_2.0/annotations.json'))
test_labels = json.load(open('AIPI540_proj1_data/raw_data/public_testing_set_release_2.0/annotations.json'))

# Create dataframe
X = pd.DataFrame(ic_train)
X = X.append(pd.DataFrame(ic_val))
X = X.append(pd.DataFrame(ic_test))

# Create dataframe for labels
y = pd.DataFrame(train_labels['annotations'])
y = y.append(pd.DataFrame(val_labels['annotations']))
y = y.append(pd.DataFrame(test_labels['annotations']))

In [ ]:
# Split data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=0)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=0)

# Scale data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Define SVC
kernels = ['linear', 'rbr', 'poly']
C_vals = [0.1, 1, 10, 100]
results = {}

for kernel in kernels:
    for C in C_vals:
        model = SVC(kernel=kernel, C=C)
        cross_val_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='accuracy').mean()
        results[(kernel, C)] = cross_val_scores

# Find best parameters
best_kernel, best_C = max(results, key=results.get)
print('Best kernel:', best_kernel)
print('Best C:', best_C)

# Train model
model = SVC(kernel=best_kernel, C=best_C)
model.fit(X_train_scaled, y_train)

# Validate model
y_pred_val = model.predict(X_val_scaled)
accuracy = accuracy_score(y_val, y_pred_val)
print('Accuracy on Validation:', accuracy)

classification_report = classification_report(y_val, y_pred_val)

In [ ]:
# Test model
y_pred_test = model.predict(X_test_scaled)
accuracy_test = accuracy_score(y_test, y_pred_test)
print('Accuracy on Test:', accuracy_test)

classification_report_test = classification_report(y_test, y_pred_test)